In [1]:
!pip install -q tensorflow tensorflow-datasets matplotlib


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


In [3]:
(train_ds, test_ds), ds_info = tfds.load(
    'rock_paper_scissors',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/incomplete.Z199FF_3.0.0/rock_paper_scissors-train.tfre…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/incomplete.Z199FF_3.0.0/rock_paper_scissors-test.tfrec…

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


In [4]:
IMG_SIZE = 128
BATCH_SIZE = 32

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train_ds = train_ds.map(format_example).batch(BATCH_SIZE).prefetch(1)
test_ds = test_ds.map(format_example).batch(BATCH_SIZE).prefetch(1)


In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_ds, validation_data=test_ds, epochs=5)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


79/79 ━━━━━━━━━━━━━━━━━━━━ 42s 500ms/step - accuracy: 0.6196 - loss: 0.9273 - val_accuracy: 0.7769 - val_loss: 0.5344
Epoch 2/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 35s 438ms/step - accuracy: 0.9914 - loss: 0.0356 - val_accuracy: 0.7124 - val_loss: 0.5738
Epoch 3/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 35s 443ms/step - accuracy: 0.9979 - loss: 0.0121 - val_accuracy: 0.7581 - val_loss: 0.6845
Epoch 4/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 38s 401ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.7876 - val_loss: 0.7250
Epoch 5/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 43s 431ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 0.7742 - val_loss: 0.7610


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('edge_ai_model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpj1u3xnhp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  138539888363280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138539887590864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138539888364048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138539888364240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138539888365008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138539888363472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138539887588368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138539888364432: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [7]:
loss, acc = model.evaluate(test_ds)
print(f"Test Accuracy: {acc:.2f}")

# Benefits
print("Edge AI Benefits: Real-time inference, offline capability, privacy preservation.")


12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 192ms/step - accuracy: 0.7933 - loss: 0.6914
Test Accuracy: 0.77
Edge AI Benefits: Real-time inference, offline capability, privacy preservation.


In [8]:
from google.colab import files
files.download('edge_ai_model.tflite')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>